# Automatización del llenado de Google Forms

Para este ejemplo usaremos un controlador web (en mi caso para el navegador Edge), Pyhton con Selenium.
El objetivo de este pequeño script es mostrar una forma de realizar la tarea de llenar formularios de Google automáticamente desde un archivo csv.

Lo primero es importar los módulos necesarios:

- Pandas: para manipular los datos.
- Selenium: para el web scrapping y controlar el navegador
- Time: para introducir algunas pausas en el llenado automático (para evitar ciertos errores)

In [7]:
import pandas as pd
from selenium import webdriver
import time

Creamos un objeto con los datos de prueba (desde un archivo csv).

In [8]:
df = pd.read_csv('datos.csv', sep=';')

En el formulario hay 2 campos que presentan ciertas dificultades para la automatización. Uno es un campo de selección única y otro es la selección desde un listado deplegable. 
Para abordar estos campos, creamos diccionarios para cada caso.

Para esta automatización es fundamental el buscar e indetificar los xpath del formulario.

In [9]:
sex_dict = {

    'Femenino': '//*[@id="i17"]/div[3]/div',
    'Masculino': '//*[@id="i20"]/div[3]/div',
    'Otro': '//*[@id="i23"]/div[3]/div'
    
}

In [10]:
category_dict = {

    'Retail': '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div[2]/div[3]',
    'Salud': '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div[2]/div[4]',
    'Educación': '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div[2]/div[5]',
    'Transporte': '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div[2]/div[6]'

}

A continuación, generamos un ciclo for para iterar cada línea del archivo de prueba e ir completando cada campo para cada formulario.
Como se mencionó antes, se han incluído algunas pausas (time.sleep) para dar tiempo a que cargue las secciones del formulario y evitar errores en la ejecución.

In [11]:
driver = webdriver.Edge() # Open web browser

for row, data in df.iterrows():
    
    name = data['Nombre']
    surname = data['Apellido paterno']
    lastname = data['Apellido materno']
    sex = data['Sexo']
    age = data['Edad']
    category = data['Rubro']

    driver.get('https://forms.gle/m8B5xc4iPLR392fg7') # Go to form link
    time.sleep(1)
    driver.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div/div[1]/input').send_keys(name) # Insert Name
    driver.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input').send_keys(surname) # Insert surname
    driver.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[3]/div/div/div[2]/div/div[1]/div/div[1]/input').send_keys(lastname) # Insert lastname
    driver.find_element('xpath', sex_dict[sex]).click() # Insert sex
    time.sleep(1)
    driver.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div[1]/div[1]/div[1]/span').click() # Open list category
    time.sleep(1)
    driver.find_element('xpath', category_dict[category]).click() # Select category
    driver.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[3]/div[1]/div[1]/div/span/span').click() # Send form

Finalmente, cerramos el controlador web.

In [12]:
driver.quit()